In [1]:
import sagemaker
import boto3
import os
import wandb
from sagemaker.pytorch import PyTorch

In [2]:
try:
    sagemaker_session = sagemaker.Session()
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_permissions')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name Shivam to get Role path.


In [3]:
from datetime import datetime

bucket = "ai-sagemaker-weights"
base_job_name = "seat-gen-unpaired"
checkpoint_suffix = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
checkpoint_s3_path = f"s3://{bucket}/{base_job_name}-{checkpoint_suffix}"
print(checkpoint_s3_path)

s3://ai-sagemaker-weights/seat-gen-unpaired-2022-08-26-18-53-05


In [4]:
source_dir = os.getcwd()
wandb.sagemaker_auth(path=source_dir)

estimator = PyTorch(entry_point='sage_train.sh',
                    source_dir=source_dir,
                    role=role,
                    py_version='py38',
                    framework_version='1.11.0',
                    base_job_name=base_job_name,
                    checkpoint_s3_uri=checkpoint_s3_path,
#                     input_mode='FastFile',
                    instance_count=1,
                    instance_type='ml.g5.xlarge',
                    use_spot_instances=True,  # Use a spot instance
                    max_run=2*24*60*60,  # Max training time
                    max_wait=2*24*60*60,  # Max training time + spot waiting time seconds
                    hyperparameters={
                        "encoder": "seresnet18",
                        "n_epochs": 100,
                        "n_epochs_decay": 100,
                        "batch_size": 16,
                        "lr": 0.0002,
                        "n_layers_D": 3,
                        "project": "new_cut",
                        "use_wandb": True,
                        "name": "no_windows/",
                    })

In [ ]:
estimator.fit({'train': 's3://ai-sagemaker-datasets/seat_gen_unpaired/'}, wait=True)

2022-08-26 13:23:22 Starting - Starting the training job...
2022-08-26 13:23:44 Starting - Preparing the instances for trainingProfilerReport-1661520200: InProgress
......
2022-08-26 13:25:05 Downloading - Downloading input data...............................................................................................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-26 13:43:23,707 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-26 13:43:23,726 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-26 13:43:23,732 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-26 13:43:24,852 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    },
    "current_host


2022-08-26 13:43:49 Training - Training image download completed. Training in progress.Successfully installed GitPython-3.1.27 PyWavelets-1.3.0 albumentations-1.2.1 docker-pycreds-0.4.0 gitdb-4.0.9 opencv-python-headless-4.6.0.66 pathtools-0.1.2 promise-2.3 qudida-0.0.4 scikit-image-0.19.3 sentry-sdk-1.9.5 setproctitle-1.3.2 shortuuid-1.0.9 smmap-5.0.0 tifffile-2022.8.12 timm-0.6.7 wandb-0.13.2
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, wor

[2022-08-26 13:44:00.320 algo-1:73 INFO state_store.py:115] Timestamps of different checkpoint files [('/opt/ml/checkpoints/no_windows/train_opt.txt', 1661521421.6218147)]
INFO:torch.distributed.distributed_c10d:Added key: store_based_barrier_key:1 to store for rank: 0
INFO:torch.distributed.distributed_c10d:Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 1 nodes.
algo-1:73:73 [0] ofi_init:1304 NCCL WARN NET/OFI Only EFA provider is supported
algo-1:73:73 [0] ofi_init:1355 NCCL WARN NET/OFI aws-ofi-nccl initialization failed
NCCL version 2.10.3+cuda11.3
/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:489: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slow